#### Connect to Abacus
You can find your API key here: [API KEY](https://abacus.ai/app/profile/apikey)

In [ ]:
import abacusai
client = abacusai.ApiClient("YOUR API KEY")

#### Finding API's easily.
There are two ways to find API's easily in the Abacus platform:
1. Try auto-completion by using tab. Most API's follow expressive language so you can search them using the autocomplete feature.
2. Use the `suggest_abacus_apis` method. This method calls a large language model that has access to our full documentation. It can suggest you what API works for what you are trying to do.
3. Use the official [Python SDK documentation](https://abacusai.github.io/api-python/autoapi/abacusai/index.html) page which will have all the available methods and attributes of classes.

In [ ]:
apis = client.suggest_abacus_apis("list feature groups in a project", verbosity=2, limit=3)
for api in apis:
    print(f"Method: {api.method}")
    print(f"Docstring: {api.docstring}")
    print("---")

#### Project Level API's
You can find the ID easily by looking at the URL in your browser. For example, if your URL looks like this: `https://abacus.ai/app/projects/fsdfasg33?doUpload=true`, the project id is "fsdfasg33"

In [ ]:
# Gets information about the project based on the ID.
project = client.describe_project(project_id="YOUR_PROJECT_ID")

# A list of all models trained under the project
models = client.list_models(project_id="YOUR_PROJECT_ID")

#### Load a Feature Group

In [ ]:
# Loads the specific version of a FeatureGroup class object 
fg = client.describe_feature_group_version("FEATURE_GROUP_VERSION")

# Loads the latest version of a FeatureGroup class object based on a name
fg = client.describe_feature_group_by_table_name("FEATURE_GROUP_NAME")

# Loads the FeatureGroup as a pandas dataframe
df = fg.load_as_pandas()

#### Add a Feature Group to the Project

In [ ]:
# First we connect our docstore to our project
client.add_feature_group_to_project(
    feature_group_id='FEATURE_GROUP_ID',
    project_id='PROJECT_ID',
    feature_group_type='CUSTOM_TABLE'  # You can set to DOCUMENTS if this is a document set
)

#### Update the feature group SQL definition

In [ ]:
client.update_feature_group_sql_definition('YOUR_FG_ID', 'SQL')

#### Creating a Dataset from local
For every dataset created, a feature group with the same name will also be generated. When you need to update the source data, just update the dataset directly and the feature group will also reflect those changes.

In [ ]:
import io
zip_filename= 'sample_data_folder.zip'

with open(zip_filename, 'rb') as f:
    zip_file_content = f.read()

zip_file_io = io.BytesIO(zip_file_content)

# If the ZIP folder contains unstructured text documents (PDF, Word, etc.), then set `is_documentset` == True
upload = client.create_dataset_from_upload(table_name='MY_SAMPLE_DATA', file_format='ZIP', is_documentset=False)
upload.upload_file(zip_file_io)

#### Updating a Dataset from local

In [ ]:
upload = client.create_dataset_version_from_upload(dataset_id='YOUR_DATASET_ID', file_format='ZIP')
upload.upload_file(zip_file_io)

#### Executing SQL using a connector

In [ ]:
connector_id = "YOUR_CONNECTOR_ID"
sql_query = "SELECT * FROM TABLE LIMIT 5"

result = client.query_database_connector(connector_id, sql_query)

#### Uploading a Dataset using a connector

`doc_processing_config` is optional depending on if you want to load a document set or no. use the code below and change based on your application. 

Similar to `create_dataset_from_file_connector` you can use `create_dataset_from_database_connector`.

In [ ]:
# doc_processing_config = abacusai.DatasetDocumentProcessingConfig(
#     extract_bounding_boxes=True,
#     use_full_ocr=False,
#     remove_header_footer=False,
#     remove_watermarks=True,
#     convert_to_markdown=False,
# )

dataset = client.create_dataset_from_file_connector(
    table_name="MY_TABLE_NAME",
    location="azure://my-location:share/whatever/*",
    # refresh_schedule="0 0 * * *",  # Daily refresh at midnight UTC
    # is_documentset=True, #Only if this is an actual documentset (Meaning word documents, PDF files, etc)
    # extract_bounding_boxes=True,
    # document_processing_config=doc_processing_config,
    # reference_only_documentset=False,
)

#### Updating a Dataset using a connector

In [ ]:
client.create_dataset_version_from_file_connector('DATASET_ID') # For file connector
client.create_dataset_version_from_database_connector('DATASET_ID')

#### Export A feature group to a connector
Below code will also work for non-SQL connectors like blob storages. The `database_feature_mapping` would be optional in those cases.

You can find the `connector_id` [here](https://abacus.ai/app/profile/connected_services)

In [ ]:
WRITEBACK = 'Anonymized_Store_Week_Result'
MAPPING = {
  'COLUMN_1': 'COLUMN_1', 
  'COLUMN_2': 'COLUMN_2', 
}

feature_group = client.describe_feature_group_by_table_name(f"FEATURE_GROUP_NAME")
feature_group.materialize() # To make sure we have latest version
feature_group_version = feature_group.latest_feature_group_version.feature_group_version
client.export_feature_group_version_to_database_connector(
    feature_group_version, 
    database_connector_id='connector_id',
    object_name=WRITEBACK,
    database_feature_mapping=MAPPING, 
    write_mode='insert'
)

#### 